In [ ]:
from notebooks.VariationNest.greedy_algo_fast import get_sparse_cluster_sums, get_sparse_p_power_sum
from randcolorgraphs.objectives.get_adjacency_dict import get_adjacency_dict
from randcolorgraphs.objectives.get_cluster_to_indices import get_cluster_to_indices
from randcolorgraphs.objectives.get_scalar_cluster_sums import get_scalar_cluster_sums
import numpy as np


@njit
def get_objective(current_clusters, x, At_sparse, w, p=2):
    cluster_to_indices = get_cluster_to_indices(current_clusters)  # O(n)
    Sx = get_scalar_cluster_sums(x, cluster_to_indices)  # O(n)
    SAt = get_sparse_cluster_sums(At_sparse, cluster_to_indices)  # O(|E|)
    p_power_sum_SAt = get_sparse_p_power_sum(SAt, p=p)  # O(|E|)

    obj = np.sum(x**p)
    for c in cluster_to_indices:
        cluster_indices = cluster_to_indices[c]
        n_j = len(cluster_indices)

        obj += -Sx[c] ** p / n_j + w * p_power_sum_SAt[c] / n_j

    return obj

def greedy_search(x, edges_out_to_in, inital_clusters, w, max_iter=100000000):
    assert np.array_equal(x, np.sort(x)), "x is not sorted!"
    assert inital_clusters.dtype.kind == "i"
    assert x.dtype.kind == "f"
    assert edges_out_to_in.dtype.kind == "i"

    n = len(x)
    edges_in_to_out = edges_out_to_in[:, ::-1]
    At_sparse = get_adjacency_dict(edges_in_to_out, n)

    print("Getting Initial Objective")
    current_clusters = inital_clusters
    current_objective = get_objective(current_clusters, x, At_sparse, w, p=2)


    cluster_states = Dict.empty(key_type=types.int64, value_type=types.int64)

    print("Starting Iterating")
    for iteration in range(max_iter):

        if best_delta_obj >= 0:
            break  # No improvement found, stop the search

        # Update clusters
        for i in best_cluster_update:
            current_clusters[i] = best_cluster_update[i]

        current_objective += best_delta_obj
        print("Iteration", iteration, "Objective:", current_objective)

    return current_clusters, current_objective


# 2 swap worst


In [1]:
from numba import njit, types
from numba.typed import Dict

# Define the type for the tuple
tuple_type = types.Tuple((types.int64, types.float64))

# Define the type for the dictionary
int_tuple_dict_type = types.DictType(types.int64, tuple_type)

@njit
def create_dict():
    # Create a typed dictionary with the specified type
    d = Dict.empty(
        key_type=types.int64,
        value_type=tuple_type
    )

    # Add some data to the dictionary
    d[1] = (10, 1.1)
    d[2] = (20, 2.2)
    d[3] = (30, 3.3)

    return d

@njit
def use_dict(d):
    # Access and print data from the dictionary
    for key in d:
        value = d[key]
        print(f"Key: {key}, Value: {value}")

# Create the dictionary
d = create_dict()

# Use the dictionary
use_dict(d)


Key: 1, Value: <object type:Tuple(int64, float64)>
Key: 2, Value: <object type:Tuple(int64, float64)>
Key: 3, Value: <object type:Tuple(int64, float64)>
